In [1]:
# 3.정적 크롤링을 활용한, 알라딘 중고서점의 데이터를 수집해서 json 파일, Maria DB저장하기.
import bs4
import csv
import urllib
import json
import pymysql

In [7]:
aladin_url = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=TodayHot&BranchType=1&CID=1'
book_data = []
for i in range(1,7):
    page = i
    page_param = f'&page={page}'
    bsObj = bs4.BeautifulSoup(urllib.request.urlopen(aladin_url+page_param).read(), 'html.parser')
    book_container = bsObj.find('form', {'name':'Myform'})
    book_name = book_container.find_all('a',class_='bo3')
    book_discounted_price = book_container.find_all('span',class_='ss_p2')
    for book,price in zip(book_name,book_discounted_price):
        title = book.text # 제목
        authors_list = book.parent.next_sibling.next_sibling.find_all('a', href=lambda x: x and x.startswith('/Search')) # 지은이
        authors = ','.join([author.text for author in authors_list])
        original_price = f"{price.previous_sibling.previous_sibling.text}원" # 원가
        discounted_price = price.text # 할인가
        print(f"책 제목:{title}\n지은이:{authors}\n원가:{original_price}\n할인가:{discounted_price}")
        book_data.append({"title":title, "author":authors, "oPrice":original_price, "dPrice":discounted_price})

책 제목:퇴마록 소장판 전권 세트 - 전17권
지은이:이우혁
원가:374,000원
할인가:336,600원
책 제목:봄밤의 모든 것
지은이:백수린
원가:17,000원
할인가:15,300원
책 제목:샤일록 작전
지은이:필립 로스,김승욱
원가:22,000원
할인가:19,800원
책 제목:해파리를 따라서 여름으로
지은이:박서형
원가:15,000원
할인가:13,500원
책 제목:나의 작은 무법자
지은이:크리스 휘타커,김해온
원가:19,000원
할인가:17,100원
책 제목:그 개와 혁명
지은이:예소연,김기태,문지혁,서장원,정기현,최민우
원가:17,500원
할인가:15,750원
책 제목:오래된 어둠과 하우스의 빛
지은이:김연덕
원가:13,000원
할인가:11,700원
책 제목:자기만의 집
지은이:전경린
원가:17,500원
할인가:15,750원
책 제목:러시아의 문장들
지은이:벨랴코프 일리야
원가:19,000원
할인가:17,100원
책 제목:창가의 토토, 그 후 이야기
지은이:구로야나기 테츠코,이와사키 치히로,권남희
원가:18,800원
할인가:16,920원
책 제목:나?
지은이:페터 플람,이창남
원가:13,000원
할인가:11,700원
책 제목:노을 진 산정에서
지은이:미나토 가나에,심정명
원가:17,800원
할인가:16,020원
책 제목:이상한 집 2
지은이:우케쓰,김은모
원가:18,000원
할인가:16,200원
책 제목:입에 대한 앙케트
지은이:세스지,오삭
원가:6,500원
할인가:5,850원
책 제목:3월의 마치
지은이:정한아
원가:16,800원
할인가:15,120원
책 제목:베리에이션 루트
지은이:마쓰나가 K 산조,김은모
원가:17,000원
할인가:15,300원
책 제목:앨리스의 모든 것
지은이:코니 윌리스,김세경
원가:16,800원
할인가:15,120원
책 제목:소년
지은이:가와바타 야스나리,정수윤
원가:16,800원
할인가:15,120원
책 제목:낭만 사랑니
지은이:청예
원가:15,000원
할인가:13,500원
책 제목:초판본 변신
지은이:프란츠 카프카,한영란


In [8]:
with open(r"Q3_hcb.json","w",encoding="utf-8",newline="") as jsonfile:
    json.dump(book_data, jsonfile, ensure_ascii=False, indent=4)

In [10]:
db = pymysql.connect(host="localhost", user="webuser", passwd="webuser", db="webdb")

cursor = db.cursor()

listed_book_data = [(book['title'],book['author'],book['oPrice'],book['dPrice']) for book in book_data]

query = "INSERT INTO booksAladin (title, author, oPrice, dPrice) VALUES (%s, %s, %s, %s)"

try:
    # execute , execute many 둘 다 tuple, list 의 형태여야 한다. 그래야 순서에 맞춰 values 에 넣을 수 있음
    # 그래서 위에서 dictionary 인 book 을 tuple 형태로 바꿔 리스트로 다시만듦
    cursor.executemany(query,listed_book_data)
    db.commit()
    print("db 저장 완료")
except Exception as e:
    db.rollback()
    print(e)
cursor.close()
db.close()

db 저장 완료
